<p align="center">
  <img width="100" height="100" src="../../images/logo.png">
</p>

<div>
<h1>022. Preproceso: Modelo C_SEV</h1> 

Canadian Car Accidents Practice <br>
<strong>Aprendizaje Automático</strong> <br>
<strong>Master Universitario en Ciencia de Datos<strong>
</div>

<div style='text-align:right'>Álvaro Serrano del Rincón (<i>a.serranodelrincon@cunef.edu</i>)</div>
<div style='text-align:right'>Carlos Viñals Guitart (<i>carlos.vinals@cunef.edu</i>)</div>

---

## 22.0 Introducción

En este notebook realizaremos el preprocesado de los datos del dataset conforme al análisis realizado en el notebook EDA 
```01_EDA```. Para ello procederemos a explicar paso a paso las decisiones tomadas en cuanto a su preprocesado.

Para este trabajo estamos utilizando un entorno de propósito espécifico.

In [1]:
# Verificamos el entorno: ML_P1
!conda info


     active environment : ML_P1
    active env location : C:\Users\carviagu\anaconda3\envs\ML_P1
            shell level : 1
       user config file : C:\Users\carviagu\.condarc
 populated config files : 
          conda version : 4.10.3
    conda-build version : 3.21.4
         python version : 3.8.8.final.0
       virtual packages : __cuda=11.3=0
                          __win=0=0
                          __archspec=1=x86_64
       base environment : C:\Users\carviagu\anaconda3  (writable)
      conda av data dir : C:\Users\carviagu\anaconda3\etc\conda
  conda av metadata url : None
           channel URLs : https://repo.anaconda.com/pkgs/main/win-64
                          https://repo.anaconda.com/pkgs/main/noarch
                          https://repo.anaconda.com/pkgs/r/win-64
                          https://repo.anaconda.com/pkgs/r/noarch
                          https://repo.anaconda.com/pkgs/msys2/win-64
                          https://repo.anaconda.com/pkgs/msys2/no

### 22.0.1 Estructura

<< PONER ESTRUCTURA >>

## Librerías

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
import matplotlib.ticker as ticker
import plotly.express as px

%matplotlib inline

from sklearn.utils import resample
from sklearn.model_selection import train_test_split

## Scripts
En este notebook procedemos a importar dos scripts con funciones útiles, previamente utilizadas en el EDA y que permitirán verificar el proceso de realización de las muestras de train y test.

In [3]:
import sys

import sys  
sys.path.insert(0, '../../scripts/')

import csv_tools
import eda_tools

## 22.1 Lectura y preparación

A continuación leeremos el dataset, de la misma forma que lo hicimos en el EDA.

In [4]:
# Función propia que verifica que existe el fichero de datos previamente.
accidents_df = csv_tools.csv_import(origin="../../data/NCDB_1999_to_2014.csv")

Reading file...
Reading ended.


In [5]:
accidents_df.head()

,C_YEAR,C_MNTH,C_WDAY,C_HOUR,C_SEV,C_VEHS,C_CONF,C_RCFG,C_WTHR,C_RSUR,...,V_ID,V_TYPE,V_YEAR,P_ID,P_SEX,P_AGE,P_PSN,P_ISEV,P_SAFE,P_USER
0,1999,01,1,20,2,02,34,UU,1,5,...,01,06,1990,01,M,41,11,1,UU,1
1,1999,01,1,20,2,02,34,UU,1,5,...,02,01,1987,01,M,19,11,1,UU,1
2,1999,01,1,20,2,02,34,UU,1,5,...,02,01,1987,02,F,20,13,2,02,2
3,1999,01,1,08,2,01,01,UU,5,3,...,01,01,1986,01,M,46,11,1,UU,1
4,1999,01,1,08,2,01,01,UU,5,3,...,99,NN,NNNN,01,M,05,99,2,UU,3


## 22.2 Balanceo de ```C_SEV```
En el EDA pudimos apreciar como había un importante desequilibrio en la variable objetivo, existiendo un 98% y 2% de accidentes no mortales y mortales respectivamente. 

In [6]:
accidents_df['C_SEV'].value_counts()

2    5761772
1      98633
Name: C_SEV, dtype: int64

Para ello realizaremos un remuestreo de los datos mediante la técnica de **subsampling** con el objetivo de reducir el número de muestras de la clase 2 (ningún fallecido) frente a las de 1 (al menos un fallecido). Para ello:
* Primero, dividimos el dataset dos dataset, uno por clase.
* Segundo, realizamos un resample de la clase mayoritaria, para extraer una muestra del mismo. 
* Finalmente, juntaremos ambos datasets para formar la muestra a utilizar. 

In [7]:
# Dividimos la clases
sev01_df = accidents_df[accidents_df['C_SEV'] == 1]
sev02_df = accidents_df[accidents_df['C_SEV'] == 2]

Realizamos una muestra de ```sev02_df``` con un tamaño superior al 5% de ```sev01_df```. Esto se realiza para que pese a reducir el desequilibrio existente, buscamos que se asemeje a la realidad y por lo tanto mantenga cierto grado de desequilibrio. 

Destacar que usamos como semilla el valor 0. Este se ha determinado de forma arbitraria, permitirá replicar los resultados en caso de querer realizar más pruebas.

In [8]:
# Tamaño a muestrear
sample_size = int(sev01_df.shape[0] * (1.05))

# Realizamos una muestra del sev02_df
sev02_sam_df = resample(sev02_df, replace=True, n_samples = sample_size, random_state=0)

# Juntamos y creamos la versión a utilizar
accidents_eq_df = pd.concat([sev01_df, sev02_sam_df])
accidents_eq_df['C_SEV'].value_counts()

2    103564
1     98633
Name: C_SEV, dtype: int64

Observamos como se ha reducido considerablemente la muestra de accidentes no mortales, y hemos podido equilibrar en cierto grado el dataset. 

Ahora, guardaremos esta versión reducida del dataset para las siguientes tareas.

In [9]:
# Guardar resultados (CUIDADO: sobreescritura del fichero)
# accidents_eq_df.to_csv("../../data/NCBD_reduced.csv")

## 22.3 Train y Test
A continuación vamos a proceder a crear y dividir los datos en Train (muestra de entrenamiento) y test (muestra de test) que usaremos para nuestros modelos. 

In [40]:
# accidents_eq_df = csv_tools.csv_import(origin = "../../data/NCBD_reduced.csv")

# Separamos los valores (X) de la variable objetivo (Y)
X = accidents_eq_df.drop('C_SEV', axis=1)
Y = accidents_eq_df['C_SEV']

# Realizamos la división de train y test
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.20, random_state=0)

In [41]:
# Juntamos para agilizar la transformación de los datos
XY_train = X_train
XY_train['C_SEV'] = Y_train

XY_test = X_test
XY_test['C_SEV'] = Y_test

## 22.4 Tratamiento
A continuación procederemos a realizar la limpieza de los datos y su procesamiento:

### 22.4.1 Filtrado inicial de variables y valores anómalos
Por otro lado, en el EDA pudimos observar que existian vehículos con fecha de fabricación 2015, cuando los accidentes son hasta 2014. Procedemos a eliminarlos en el caso de que continuen en la muestra de train.

In [42]:
XY_train = XY_train.drop(XY_train[XY_train['V_YEAR'] == '2015'].index, axis=0)
XY_test = XY_test.drop(XY_test[XY_test['V_YEAR'] == '2015'].index, axis=0)

Por otro lado eliminamos las filas que hacen referencia a coches aparcados, es decir, cuyo valor edad de la persona (```P_AGE```) es ```NN```, es decir, no aplicable al no ser una persona. 

In [43]:
XY_train = XY_train.drop(XY_train[XY_train['P_AGE'] == 'NN'].index, axis=0)
XY_test = XY_test.drop(XY_test[XY_test['P_AGE'] == 'NN'].index, axis=0)

Si bien nos detendremos en datalle a analizar la importancia de las variables para el modelo. Existen a priori unas variables que no son útiles y que por lo tanto eliminaremos del mismo. Queremos informacion relativa a un accidente, como analizamos en el EDA en este caso el dataset hace referencia a la situación de una persona en una accidente. Por lo que eliminamos todas las variables sobre personas y vehículos.

Primero arreglaremos la variable EDAD de la persona que tiene valores nulos. En este caso los sustituiremos por la edad media calculada del conjunto de datos existente:

In [44]:
# Tratamiento inicial de variables para agregación de edades
# TRAIN
XY_train['P_AGE'] = XY_train['P_AGE'].replace('UU', np.nan)
XY_train['P_AGE'] = XY_train['P_AGE'].astype(float)
mean_age = np.round(XY_train['P_AGE'].mean(), decimals = 0) 
XY_train['P_AGE'] = XY_train['P_AGE'].replace(np.nan, mean_age)

# TEST
XY_test['P_AGE'] = XY_test['P_AGE'].replace('UU', np.nan)
XY_test['P_AGE'] = XY_test['P_AGE'].astype(float)
mean_age = np.round(XY_test['P_AGE'].mean(), decimals = 0) 
XY_test['P_AGE'] = XY_test['P_AGE'].replace(np.nan, mean_age)

A continuación, realizaremos la agrupación por accidentes. Agregaremos el número de personas implicadas (número de filas con la misma información de accidente) y la edad media de los involucrados en el accidente:

In [45]:
# Agregación final
acc_columns = ['C_YEAR', 'C_MNTH', 'C_WDAY', 
               'C_HOUR', 'C_SEV', 'C_VEHS', 
               'C_CONF', 'C_RCFG', 'C_WTHR', 
               'C_RSUR', 'C_RALN', 'C_TRAF']

XY_train = XY_train.groupby(acc_columns).agg(C_PERS=('P_ID', 'count'), C_AGES=('P_AGE', 'mean')).reset_index()
XY_test = XY_test.groupby(acc_columns).agg(C_PERS=('P_ID', 'count'), C_AGES=('P_AGE', 'mean')).reset_index()

Analizamos tipos:

In [46]:
eda_tools.dataset_overview(data = XY_train)

,columna,tipo_dato,nulos,nulos%
0,C_YEAR,int64,0,0.0
1,C_MNTH,object,0,0.0
2,C_WDAY,object,0,0.0
3,C_HOUR,object,0,0.0
4,C_SEV,int64,0,0.0
5,C_VEHS,object,0,0.0
6,C_CONF,object,0,0.0
7,C_RCFG,object,0,0.0
8,C_WTHR,object,0,0.0
9,C_RSUR,object,0,0.0


### 22.4.2 Valores faltantes
Para llevar a cabo este proceso realizaremos la conversión correspondiente por tipo de variable, esto es, pues el signficado que estas tienen es distinto. 

A nivel general si el porcentaje es inferior a 0.1 dedicidimos eliminar esas variables, en caso contrario aplicaremos una técnicas de sustitución de ese valor. Consideramos que un valor inferior a 0.1 no es relevante para los datos.

Primero veamos como está la muestra de train:

In [47]:
especial_values = [['U', 'UU', 'UUUU'], ['N', 'NN', 'NNNN'], ['Q', 'QQ', 'QQQQ']]

# Valoración del train
eda_tools.special_values_summary(df = XY_train, vals = especial_values)

,column_name,U,U%,N,N%,Q,Q%,total,total%
0,C_YEAR,0,[0.0],0,[0.0],0,[0.0],0,0.0
1,C_MNTH,5,[0.004],0,[0.0],0,[0.0],5,0.0
2,C_WDAY,22,[0.019],0,[0.0],0,[0.0],22,0.0
3,C_HOUR,1260,[1.113],0,[0.0],0,[0.0],1260,1.1
4,C_SEV,0,[0.0],0,[0.0],0,[0.0],0,0.0
5,C_VEHS,8,[0.007],0,[0.0],0,[0.0],8,0.0
6,C_CONF,3173,[2.804],0,[0.0],5711,[5.047],8884,7.9
7,C_RCFG,8924,[7.886],0,[0.0],3242,[2.865],12166,10.8
8,C_WTHR,1508,[1.333],0,[0.0],310,[0.274],1818,1.6
9,C_RSUR,1368,[1.209],0,[0.0],3334,[2.946],4702,4.2


In [57]:
# Valoración del test
eda_tools.special_values_summary(df = XY_test, vals = especial_values)

,column_name,U,U%,N,N%,Q,Q%,total,total%
0,C_YEAR,0,[0.0],0,[0.0],0,[0.0],0,0.0
1,C_MNTH,1,[0.003],0,[0.0],0,[0.0],1,0.0
2,C_WDAY,4,[0.011],0,[0.0],0,[0.0],4,0.0
3,C_HOUR,0,[0.0],0,[0.0],0,[0.0],0,0.0
4,C_SEV,0,[0.0],0,[0.0],0,[0.0],0,0.0
5,C_VEHS,6,[0.017],0,[0.0],0,[0.0],6,0.0
6,C_CONF,945,[2.658],0,[0.0],1845,[5.19],2790,7.8
7,C_RCFG,2667,[7.503],0,[0.0],989,[2.782],3656,10.3
8,C_WTHR,401,[1.128],0,[0.0],107,[0.301],508,1.4
9,C_RSUR,378,[1.063],0,[0.0],1075,[3.024],1453,4.1


**Para los valores U (desconocido)**: Se establecerá el valor medio ponderado (redondeado a la unidad superior) de las categorías asignadas. Al haber tanta variabilidad asignaremos la categoría media.

**Para los valores Q (otra categoría no contemplada)**: Se eliminarán del dataset pues no poseen información útil de cara a determinar la mortalidad.

Es importante tener en cuenta aspectos de la variable que estemos hablando, lo vemos  a continuación:

```C_MNTH```: Solo tiene valores U, es decir, NaN. Observamos que su valor es inferior a 0.1, son solo 5 filas, por lo que las eliminamos al no tener un peso significativo.

In [48]:
eda_tools.classes_overview(df = XY_train, obj_val = 'C_MNTH')

,index,percentaje,C_MNTH
0,08,9.595094,10858
1,07,9.499655,10750
2,09,9.093158,10290
3,10,9.052509,10244
4,12,8.901398,10073
5,06,8.829819,9992
6,11,8.691080,9835
7,01,8.113148,9181
8,05,7.944363,8990
9,02,6.841519,7742


In [58]:
XY_train = XY_train.drop(XY_train[XY_train['C_MNTH'] == 'UU'].index, axis=0)
XY_test = XY_test.drop(XY_test[XY_test['C_MNTH'] == 'UU'].index, axis=0)

```C_WDAY```: Solo tiene valores U:

In [50]:
eda_tools.classes_overview(df = XY_train, obj_val = 'C_WDAY')

,index,percentaje,C_WDAY
0,5,16.826180,19040
1,6,15.420168,17449
2,4,14.610674,16533
3,3,13.596154,15385
4,2,13.583782,15371
5,1,13.197593,14934
6,7,12.750426,14428
7,U,0.015023,17


In [60]:
XY_train = XY_train.drop(XY_train[XY_train['C_WDAY'] == 'U'].index, axis=0)
XY_test = XY_test.drop(XY_test[XY_test['C_WDAY'] == 'U'].index, axis=0)

```C_HOUR```: Valores U

In [52]:
hour_ovw = eda_tools.classes_overview(df = XY_train[XY_train['C_HOUR'] != 'UU' ], obj_val = 'C_HOUR')
eda_tools.classes_overview(df = XY_train, obj_val = 'C_HOUR')

,index,percentaje,C_HOUR
0,16,7.863709,8897
1,17,7.765600,8786
2,15,7.659537,8666
3,14,6.301927,7130
4,18,6.253314,7075
5,13,5.783984,6544
6,12,5.718579,6470
7,11,4.944317,5594
8,19,4.906311,5551
9,08,4.632314,5241


In [53]:
hour_ovw['cat'] = hour_ovw['index'].astype(int)
hour_ovw['weight'] = hour_ovw['cat'] * hour_ovw['percentaje']
mean_val = np.round(hour_ovw['weight'].sum(axis=0) / 100, decimals = 0)
XY_train['C_HOUR'] = XY_train['C_HOUR'].replace('UU', '13')
XY_test['C_HOUR'] = XY_test['C_HOUR'].replace('UU', '13')

```C_VEHS```: Valores U

In [54]:
eda_tools.classes_overview(df = XY_train, obj_val = 'C_VEHS')

,index,percentaje,C_VEHS
0,02,56.534382,63963
1,01,29.247835,33091
2,03,10.496730,11876
3,04,2.610041,2953
4,05,0.665547,753
5,06,0.200636,227
6,07,0.078664,89
7,08,0.039774,45
8,09,0.027400,31
9,10,0.015909,18


In [61]:
XY_train = XY_train.drop(XY_train[XY_train['C_VEHS'] == 'UU'].index, axis=0)
XY_test = XY_test.drop(XY_test[XY_test['C_VEHS'] == 'UU'].index, axis=0)

```C_CONF```: Tiene valores U y Q

In [68]:
conf_ovw = eda_tools.classes_overview(df = XY_train[(XY_train['C_CONF'] != 'UU') & 
                                                    (XY_train['C_CONF'] != 'QQ')], obj_val = 'C_CONF')
eda_tools.classes_overview(df = XY_train, obj_val = 'C_CONF')

,index,percentaje,C_CONF
0,21,22.777414,25769
1,35,13.455725,15223
2,06,10.869411,12297
3,31,7.765128,8785
4,33,6.257182,7079
5,36,5.892128,6666
6,04,5.663196,6407
7,QQ,5.046228,5709
8,03,4.632560,5241
9,02,4.129616,4672


In [71]:
# Sustiuimos UU por la media ponderada
conf_ovw['cat'] = conf_ovw['index'].astype(int)
conf_ovw['weight'] = conf_ovw['cat'] * conf_ovw['percentaje']
mean_val = np.round(conf_ovw['weight'].sum(axis=0) / 100, decimals = 0)
XY_train['C_CONF'] = XY_train['C_CONF'].replace('UU', '21')
XY_test['C_CONF'] = XY_test['C_CONF'].replace('UU', '21')

# Eliminamos las variables QQ pues no aportan información util al referirse a categorías 
# no contempladas.
XY_train = XY_train.drop(XY_train[XY_train['C_CONF'] == 'QQ'].index, axis=0)
XY_test = XY_test.drop(XY_test[XY_test['C_CONF'] == 'QQ'].index, axis=0)

```C_RCFG```: Tiene valores U y Q

In [73]:
rcfg_ovw = eda_tools.classes_overview(df = XY_train[(XY_train['C_RCFG'] != 'UU') & 
                                                    (XY_train['C_RCFG'] != 'QQ')], obj_val = 'C_RCFG')
eda_tools.classes_overview(df = XY_train, obj_val = 'C_RCFG')

,index,percentaje,C_RCFG
0,01,41.994880,45113
1,02,41.163603,44220
2,UU,7.996276,8590
3,03,4.478473,4811
4,QQ,2.464976,2648
5,05,0.979288,1052
6,04,0.563184,605
7,08,0.182453,196
8,06,0.115429,124
9,07,0.030719,33


In [76]:
# Sustiuimos UU por la media ponderada
rcfg_ovw['cat'] = rcfg_ovw['index'].astype(int)
rcfg_ovw['weight'] = rcfg_ovw['cat'] * rcfg_ovw['percentaje']
mean_val = np.round(rcfg_ovw['weight'].sum(axis=0) / 100, decimals = 0)
XY_train['C_RCFG'] = XY_train['C_RCFG'].replace('UU', '02')
XY_test['C_RCFG'] = XY_test['C_RCFG'].replace('UU', '02')

# Eliminamos las variables QQ pues no aportan información util al referirse a categorías 
# no contempladas.
XY_train = XY_train.drop(XY_train[XY_train['C_RCFG'] == 'QQ'].index, axis=0)
XY_test = XY_test.drop(XY_test[XY_test['C_RCFG'] == 'QQ'].index, axis=0)

```C_WTHR```: Tiene valores U y Q

In [ ]:
rcfg_ovw = eda_tools.classes_overview(df = XY_train[(XY_train['C_RCFG'] != 'UU') & 
                                                    (XY_train['C_RCFG'] != 'QQ')], obj_val = 'C_RCFG')
eda_tools.classes_overview(df = XY_train, obj_val = 'C_RCFG')

In [22]:
# VALORES EXTRAÑOS
# Eliminar valores extraños (año 2015?)

In [23]:
# ENCODING ¿?
# Sexo -> M/F LabelEncoding o OneHotEncoding
# 
# Encoding de variables con una codificación extraña  
#

In [24]:
# SELECCIÓN DE VARIABLES
# Correlación ?
# Lasso o modelo de regresión?
# Justificar elección final ? 

---

<div style='text-align:center'>Elaborado por Álvaro Serrano del Rincón (<i>a.serranodelrincon@cunef.edu</i>)</div> 
<div style='text-align:center'>y Carlos Viñals Guitart (<i>carlos.vinals@cunef.edu</i>)</div> 